# Calculation of contact angles with cDFT

In [1]:
import feos
import si_units as si
import numpy as np

# set up fluid model
parameters = feos.Parameters.from_json(["methanol"], "../../parameters/pcsaft/esper2023.json")
functional = feos.HelmholtzEnergyFunctional.pcsaft(parameters)

# set up surface
ext_pot = feos.ExternalPotential.LJ93(sigma_ss=3.0, epsilon_k_ss=120.0, rho_s=0.08)
pore = feos.Pore1D(feos.Geometry.Cartesian, 50*si.ANGSTROM, ext_pot)

In [2]:
# calculate the density profiles of the vapor and the liquid states
vle = feos.PhaseEquilibrium.pure(functional, 350 * si.KELVIN)
vapor_profile = pore.initialize(vle.vapor).solve()
liquid_profile = pore.initialize(vle.liquid).solve()

# calculate the solid/fluid interfacial tensions
gamma_sl = liquid_profile.interfacial_tension / si.ANGSTROM**2
gamma_sv = vapor_profile.interfacial_tension / si.ANGSTROM**2

# calculate the vapor/liquid interfacial tension
vle_profile = feos.PlanarInterface.from_pdgt(vle, 2048).solve()
gamma_vl = vle_profile.surface_tension

# calculate the contact angle from Young's equation
np.acos((gamma_sv-gamma_sl)/gamma_vl)*si.RADIANS

31.990466526671963°